In [19]:
import os
import random
import shutil

data_dir = 'PetImages'
cat_dir = os.path.join(data_dir, 'Cat')
dog_dir = os.path.join(data_dir, 'Dog')

#Aprasom kokia dalis tenka mokymui ir testavimui
splits = (0.7, 0.15, 0.15)

def split_data(directory:str, splits:tuple):
    """
    Funkcija yra skirta suskaidyti pateiktam kataloge esancias nuotraukas i tris naujus katalogus, pagal 
    pateiktus isskaidymo dydzius

    Parametrai:
    directory - nuoroda iki failo, kuri norite skaidyti
    splits - tuple, su nurodytais kiekiais mokymui, testavimui ir validacijai
    """
    images = os.listdir(directory)
    random.shuffle(images)
    train_size = int(len(images) * splits[0])
    validation_size = int(len(images) * splits[1])
    print(validation_size)

    train_dir = os.path.join(directory, 'train')
    validation_dir = os.path.join(directory, 'validation')
    test_dir = os.path.join(directory, 'test')

    
    
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for i, image in enumerate(images):
        if i < train_size:
            shutil.copy(os.path.join(directory, image), os.path.join(train_dir, image))
        elif i < train_size + validation_size:
            shutil.copy(os.path.join(directory, image), os.path.join(validation_dir, image))
        else:
            shutil.copy(os.path.join(directory, image), os.path.join(test_dir, image))


split_data(cat_dir, splits)


1873


In [29]:
from PIL import Image

data_dir = 'PetImages'
cat_dir = os.path.join(data_dir, 'Cat')
dog_dir = os.path.join(data_dir, 'Dog')

def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError):
        return False
    
def get_valid_image_files(directory):
    """Skirta atrinkti į sąrašą tik validžių failų pavadinimus"""
    valid_files = []
    for root, _, files in os.walk(directory): # naudojame _ katalogams train, validation, test, nes neketiname jų naudoti
        if root != directory:
            for file in files:
                file_path = os.path.join(root, file)
                if is_valid_image(file_path):
                    valid_files.append(file_path)
    return valid_files

valid_cat_photos = get_valid_image_files(cat_dir)

In [30]:
print(len(valid_cat_photos))
valid_dog_photos = get_valid_image_files(dog_dir)

12489


c:\Users\sarun\OneDrive\Documents\Code\Testinei\mokymai\Lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [31]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

dataGen = ImageDataGenerator(rescale=1./255)

cat_df = pd.DataFrame({'filename':valid_cat_photos})
dog_df = pd.DataFrame({'filename':valid_dog_photos})

cat_generator = dataGen.flow_from_dataframe(
    dataframe = cat_df,
    x_col = 'filename',
    target_size = (150, 150),
    batch_size = 20,
    class_mode = None)

dog_generator = dataGen.flow_from_dataframe(
    dataframe = dog_df,
    x_col = 'filename',
    target_size = (150, 150),
    batch_size = 20,
    class_mode = None)

Found 12489 validated image filenames.
Found 12499 validated image filenames.


In [34]:
from tensorflow.keras.utils import Sequence
import numpy as np

class CombinedGenerator(Sequence):
    def __init__(self, *generators):
        self.generators = generators
        self._num_batches = sum(len(gen) for gen in generators)
        self.current_generator = 0

    def __len__(self):
        return self._num_batches
    
    def __getitem__(self, idx):
        for gen in self.generators:
            if idx < len(gen):
                batch = gen[idx]
                labels = np.array([0] * batch.shape[0]) if gen == cat_generator else np.array([1]* batch.shape[0])
                return batch, labels
            idx -= len(gen)

combined_generator = CombinedGenerator(cat_generator, dog_generator)

In [40]:
from tensorflow.keras import datasets, layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (150,150,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation = 'relu' ),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation = 'relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [41]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [62]:
history = model.fit(
    combined_generator,
    steps_per_epoch = len(combined_generator),
    epochs = 10
)

Epoch 1/10
1094/1250 ━━━━━━━━━━━━━━━━━━━━ 27s 178ms/step - accuracy: 0.6641 - loss: 0.6229

c:\Users\sarun\OneDrive\Documents\Code\Testinei\mokymai\Lib\site-packages\PIL\TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 214s 171ms/step - accuracy: 0.6653 - loss: 0.6211
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 6us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/10


C:\Users\sarun\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 219s 174ms/step - accuracy: 0.7394 - loss: 0.5259
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 5us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 271s 216ms/step - accuracy: 0.8054 - loss: 0.4162
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 167s 133ms/step - accuracy: 0.8759 - loss: 0.2926
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 128s 102ms/step - accuracy: 0.9517 - loss: 0.1342
Epoch 10/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00


In [63]:
def preprocess_image(image_path):
    img = Image.open(image_path)
    img = img.resize((150, 150))
    img_array = np.array(img)
    img_array = img_array.astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

preprocess_image('cat.jpg')

array([[[[0.7607843 , 0.76862746, 0.81960785],
         [0.8156863 , 0.83137256, 0.8784314 ],
         [0.85490197, 0.87058824, 0.91764706],
         ...,
         [0.4745098 , 0.40392157, 0.3647059 ],
         [0.4627451 , 0.39215687, 0.36078432],
         [0.45490196, 0.3882353 , 0.36078432]],

        [[0.77254903, 0.78039217, 0.83137256],
         [0.8039216 , 0.8156863 , 0.8627451 ],
         [0.83137256, 0.84705883, 0.89411765],
         ...,
         [0.4745098 , 0.40392157, 0.3647059 ],
         [0.4627451 , 0.39215687, 0.36078432],
         [0.45490196, 0.3882353 , 0.36078432]],

        [[0.78431374, 0.7921569 , 0.8392157 ],
         [0.7882353 , 0.79607844, 0.84313726],
         [0.80784315, 0.81960785, 0.8627451 ],
         ...,
         [0.47843137, 0.40784314, 0.36862746],
         [0.4627451 , 0.4       , 0.36862746],
         [0.4627451 , 0.39607844, 0.36862746]],

        ...,

        [[0.8156863 , 0.8235294 , 0.88235295],
         [0.79607844, 0.8039216 , 0.8627451 ]

In [65]:
def predict_image(image_path):
    img_array = preprocess_image(image_path)
    prediction = model.predict(img_array)
    return 'Suo' if prediction[0][0] > 0.5 else 'Kate'

print(predict_image('cat.jpg'))
print(predict_image('dog.jpg'))
print(predict_image('cat2.jpg'))
print(predict_image('dog2.jpg'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Kate
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Suo
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Kate
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Suo
